# Permute Hetnets for Interpreting Compressed Latent Spaces

Modified from @dhimmel - https://github.com/dhimmel/integrate/blob/master/permute.ipynb

Generate several randomly permuted hetnets to serve as a null distribution. The permutations preserve node degree but randomizes connections between nodes. See [Himmelstein et al. 2017](https://doi.org/10.7554/eLife.26726) for more details.

In [1]:
import os
import pandas as pd

import hetio.readwrite
import hetio.permute

In [2]:
%%time
hetnet_path = os.path.join('hetnets', 'interpret_hetnet.json.bz2')
graph = hetio.readwrite.read_graph(hetnet_path)

CPU times: user 57.8 s, sys: 1.67 s, total: 59.4 s
Wall time: 59.3 s


In [3]:
# Selected as a result of `scripts/evaluate-permutations.ipynb`
num_permuted_hetnets = 10
num_swaps = 4

In [4]:
%%time
stat_dfs = list()
permuted_graph = graph
 
for i in range(num_permuted_hetnets):
    i += 1
    print('Starting permutation', i)
    permuted_graph, stats = hetio.permute.permute_graph(permuted_graph,
                                                        multiplier=num_swaps,
                                                        seed=i)
    stat_df = pd.DataFrame(stats)
    stat_df['permutation'] = i
    stat_dfs.append(stat_df)
    perm_path = os.path.join('hetnets', 'permuted',
                             'interpret_hetnet_perm-{}.json.bz2'.format(i))
    hetio.readwrite.write_graph(permuted_graph, perm_path)

Starting permutation 1
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
CPU times: user 7h 1min 22s, sys: 3.6 s, total: 7h 1min 25s
Wall time: 7h 7s


In [5]:
# Save stats
stat_df = pd.concat(stat_dfs)
stat_path = os.path.join('hetnets', 'permuted', 'stats.tsv')
stat_df.to_csv(stat_path, sep='\t', index=False, float_format='%.5g')